# Backtest Comparison: Value-Only vs Multi-Factor Strategy

This notebook compares two investment strategies over 5 years (Jan 2021 - Jan 2026) with $1000/year investment.

## Strategies:
1. **Value-Only**: US Large Cap, Positive P/E, P/E < 0.5 × Market Cap (billions), Top 25 by market cap
2. **Multi-Factor**: Same base + EPS Growth > 0% + 6M Performance > 0%, Top 25 by composite score
3. **Benchmark**: S&P 500 (SPY ETF)

## Section 1: Setup & Data Loading

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuration
START_DATE = "2021-01-01"
END_DATE = "2026-01-22"
ANNUAL_INVESTMENT = 1000.0
NUM_STOCKS = 25

# Rebalancing dates (first trading day of each year)
REBALANCE_DATES = [
    "2021-01-04",
    "2022-01-03",
    "2023-01-03",
    "2024-01-02",
    "2025-01-02",
]

print("Configuration loaded.")

In [ ]:
# Load S&P 500 constituents from existing file or fetch from Wikipedia
try:
    sp500_df = pd.read_excel("s&p500-constituents.xlsx")
    sp500_tickers = sp500_df['Symbol'].tolist() if 'Symbol' in sp500_df.columns else sp500_df.iloc[:, 0].tolist()
    print(f"Loaded {len(sp500_tickers)} tickers from local file")
except Exception as e:
    print(f"Loading from Wikipedia...")
    sp500_table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
    sp500_tickers = sp500_table['Symbol'].str.replace('.', '-', regex=False).tolist()
    print(f"Loaded {len(sp500_tickers)} tickers from Wikipedia")

# Clean tickers (replace . with - for yfinance compatibility)
sp500_tickers = [t.replace('.', '-') for t in sp500_tickers]
print(f"Sample tickers: {sp500_tickers[:10]}")

In [ ]:
# Download historical price data for all S&P 500 stocks
print(f"Downloading price data from {START_DATE} to {END_DATE}...")
print("This may take a few minutes...")

# Download in batches to avoid timeout issues
all_prices = pd.DataFrame()
batch_size = 50

for i in range(0, len(sp500_tickers), batch_size):
    batch = sp500_tickers[i:i+batch_size]
    try:
        data = yf.download(batch, start=START_DATE, end=END_DATE, progress=False)['Adj Close']
        if isinstance(data, pd.Series):
            data = data.to_frame(batch[0])
        all_prices = pd.concat([all_prices, data], axis=1)
        print(f"Downloaded batch {i//batch_size + 1}/{(len(sp500_tickers)-1)//batch_size + 1}")
    except Exception as e:
        print(f"Error downloading batch {i//batch_size + 1}: {e}")

# Remove duplicates and fill forward missing values
all_prices = all_prices.loc[:, ~all_prices.columns.duplicated()]
all_prices = all_prices.ffill()

print(f"\nDownloaded price data for {len(all_prices.columns)} stocks")
print(f"Date range: {all_prices.index[0]} to {all_prices.index[-1]}")

In [ ]:
# Download SPY for benchmark comparison
print("Downloading SPY benchmark data...")
spy_data = yf.download("SPY", start=START_DATE, end=END_DATE, progress=False)['Adj Close']
print(f"SPY data: {len(spy_data)} trading days")

In [ ]:
# Get fundamental data for all stocks (current snapshot - we'll use as proxy)
# Note: Historical fundamentals are approximated using price-derived metrics
print("Fetching fundamental data...")

fundamentals = {}
failed_tickers = []

for i, ticker in enumerate(sp500_tickers):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        fundamentals[ticker] = {
            'marketCap': info.get('marketCap', 0),
            'trailingPE': info.get('trailingPE', None),
            'forwardPE': info.get('forwardPE', None),
            'earningsGrowth': info.get('earningsGrowth', None),
            'revenueGrowth': info.get('revenueGrowth', None),
            'sector': info.get('sector', 'Unknown'),
        }
    except Exception as e:
        failed_tickers.append(ticker)
    
    if (i + 1) % 50 == 0:
        print(f"Processed {i+1}/{len(sp500_tickers)} stocks...")

fundamentals_df = pd.DataFrame(fundamentals).T
print(f"\nGot fundamentals for {len(fundamentals_df)} stocks")
print(f"Failed: {len(failed_tickers)} stocks")

## Section 2: Strategy Implementation Functions

In [ ]:
def calculate_momentum(prices_df, ticker, date, lookback_days=126):
    """Calculate 6-month momentum (approximately 126 trading days)"""
    try:
        # Get prices up to the given date
        prices = prices_df[ticker].loc[:date]
        if len(prices) < lookback_days:
            return None
        
        current_price = prices.iloc[-1]
        past_price = prices.iloc[-lookback_days]
        
        if past_price > 0:
            return (current_price - past_price) / past_price
        return None
    except:
        return None

def estimate_pe_ratio(prices_df, fundamentals_df, ticker, date):
    """
    Estimate historical P/E ratio.
    Uses current P/E as base and adjusts by price change ratio.
    """
    try:
        current_pe = fundamentals_df.loc[ticker, 'trailingPE']
        if pd.isna(current_pe) or current_pe is None or current_pe <= 0:
            return None
        
        # Get current and historical prices
        current_price = prices_df[ticker].iloc[-1]
        hist_price = prices_df[ticker].loc[:date].iloc[-1]
        
        # Estimate historical P/E (assuming earnings grew proportionally)
        # This is a simplification - actual historical P/E would require historical earnings data
        price_ratio = hist_price / current_price if current_price > 0 else 1
        
        # Adjust P/E by price change (rough approximation)
        estimated_pe = current_pe * price_ratio
        return estimated_pe if estimated_pe > 0 else None
    except:
        return None

def estimate_market_cap(prices_df, fundamentals_df, ticker, date):
    """
    Estimate historical market cap based on price change.
    """
    try:
        current_cap = fundamentals_df.loc[ticker, 'marketCap']
        if pd.isna(current_cap) or current_cap is None or current_cap <= 0:
            return None
        
        # Get current and historical prices
        current_price = prices_df[ticker].iloc[-1]
        hist_price = prices_df[ticker].loc[:date].iloc[-1]
        
        # Estimate historical market cap
        price_ratio = hist_price / current_price if current_price > 0 else 1
        estimated_cap = current_cap * price_ratio
        return estimated_cap
    except:
        return None

print("Strategy functions defined.")

In [ ]:
def select_stocks_value(prices_df, fundamentals_df, date, num_stocks=25):
    """
    Value-Only Strategy Selection:
    1. US Large Cap ($10B+)
    2. Positive P/E
    3. P/E < 0.5 × Market Cap (in billions)
    4. Top 25 by market cap
    """
    candidates = []
    
    for ticker in prices_df.columns:
        if ticker not in fundamentals_df.index:
            continue
        
        # Check if we have price data up to this date
        try:
            price = prices_df[ticker].loc[:date].iloc[-1]
            if pd.isna(price):
                continue
        except:
            continue
        
        # Estimate historical metrics
        market_cap = estimate_market_cap(prices_df, fundamentals_df, ticker, date)
        pe_ratio = estimate_pe_ratio(prices_df, fundamentals_df, ticker, date)
        
        if market_cap is None or pe_ratio is None:
            continue
        
        market_cap_billions = market_cap / 1e9
        
        # Apply filters
        # 1. Large Cap ($10B+)
        if market_cap_billions < 10:
            continue
        
        # 2. Positive P/E
        if pe_ratio <= 0:
            continue
        
        # 3. P/E < 0.5 × Market Cap (billions)
        if pe_ratio >= 0.5 * market_cap_billions:
            continue
        
        candidates.append({
            'ticker': ticker,
            'market_cap': market_cap,
            'pe_ratio': pe_ratio,
            'price': price
        })
    
    # Sort by market cap and take top N
    candidates_df = pd.DataFrame(candidates)
    if len(candidates_df) == 0:
        return []
    
    candidates_df = candidates_df.sort_values('market_cap', ascending=False)
    selected = candidates_df.head(num_stocks)
    
    return selected.to_dict('records')

print("Value-Only selection function defined.")

In [ ]:
def select_stocks_multifactor(prices_df, fundamentals_df, date, num_stocks=25):
    """
    Multi-Factor Strategy Selection:
    1. Same value filters as above
    2. EPS Growth > 0% (quality)
    3. 6-Month Performance > 0% (momentum)
    4. Rank by composite score (Value + Growth + Momentum ranks)
    5. Top 25 by composite score
    """
    candidates = []
    
    for ticker in prices_df.columns:
        if ticker not in fundamentals_df.index:
            continue
        
        # Check if we have price data up to this date
        try:
            price = prices_df[ticker].loc[:date].iloc[-1]
            if pd.isna(price):
                continue
        except:
            continue
        
        # Estimate historical metrics
        market_cap = estimate_market_cap(prices_df, fundamentals_df, ticker, date)
        pe_ratio = estimate_pe_ratio(prices_df, fundamentals_df, ticker, date)
        momentum = calculate_momentum(prices_df, ticker, date)
        
        # Get earnings growth (using current as proxy)
        earnings_growth = fundamentals_df.loc[ticker, 'earningsGrowth']
        
        if market_cap is None or pe_ratio is None or momentum is None:
            continue
        
        market_cap_billions = market_cap / 1e9
        
        # Apply value filters
        if market_cap_billions < 10:
            continue
        if pe_ratio <= 0:
            continue
        if pe_ratio >= 0.5 * market_cap_billions:
            continue
        
        # Apply quality filter: EPS Growth > 0
        if pd.isna(earnings_growth) or earnings_growth is None or earnings_growth <= 0:
            continue
        
        # Apply momentum filter: 6-month performance > 0
        if momentum <= 0:
            continue
        
        candidates.append({
            'ticker': ticker,
            'market_cap': market_cap,
            'pe_ratio': pe_ratio,
            'earnings_growth': earnings_growth,
            'momentum': momentum,
            'price': price
        })
    
    if len(candidates) == 0:
        return []
    
    candidates_df = pd.DataFrame(candidates)
    
    # Calculate composite score using ranks
    # Value rank: lower P/E is better (ascending rank)
    candidates_df['value_rank'] = candidates_df['pe_ratio'].rank(ascending=True)
    
    # Growth rank: higher earnings growth is better (descending rank)
    candidates_df['growth_rank'] = candidates_df['earnings_growth'].rank(ascending=False)
    
    # Momentum rank: higher momentum is better (descending rank)
    candidates_df['momentum_rank'] = candidates_df['momentum'].rank(ascending=False)
    
    # Composite score (lower is better)
    candidates_df['composite_score'] = (
        candidates_df['value_rank'] + 
        candidates_df['growth_rank'] + 
        candidates_df['momentum_rank']
    )
    
    # Sort by composite score and take top N
    candidates_df = candidates_df.sort_values('composite_score', ascending=True)
    selected = candidates_df.head(num_stocks)
    
    return selected.to_dict('records')

print("Multi-Factor selection function defined.")

## Section 3: Backtest Engine

In [ ]:
def run_backtest(strategy_func, prices_df, fundamentals_df, 
                 annual_investment=1000.0, rebalance_dates=REBALANCE_DATES, 
                 num_stocks=25, strategy_name="Strategy"):
    """
    Run a backtest for the given strategy.
    
    For each rebalance date:
    1. Call strategy_func to get stock selections
    2. Allocate investment equally across selected stocks
    3. Track holdings until next rebalance
    4. At rebalance: sell all, apply new selections, reinvest total value + new investment
    
    Returns: DataFrame with daily portfolio values and transaction history
    """
    portfolio_history = []
    transaction_history = []
    
    holdings = {}  # ticker -> shares
    cash = 0.0
    total_invested = 0.0
    
    # Get all trading days
    all_dates = prices_df.index.sort_values()
    
    for i, rebal_date_str in enumerate(rebalance_dates):
        rebal_date = pd.Timestamp(rebal_date_str)
        
        # Find the actual trading day (closest date >= rebalance date)
        valid_dates = all_dates[all_dates >= rebal_date]
        if len(valid_dates) == 0:
            continue
        actual_rebal_date = valid_dates[0]
        
        # Sell all existing holdings
        if holdings:
            for ticker, shares in holdings.items():
                try:
                    sell_price = prices_df.loc[actual_rebal_date, ticker]
                    if not pd.isna(sell_price):
                        cash += shares * sell_price
                        transaction_history.append({
                            'date': actual_rebal_date,
                            'action': 'SELL',
                            'ticker': ticker,
                            'shares': shares,
                            'price': sell_price,
                            'value': shares * sell_price
                        })
                except:
                    pass
            holdings = {}
        
        # Add annual investment
        cash += annual_investment
        total_invested += annual_investment
        
        # Select stocks using strategy
        selected = strategy_func(prices_df, fundamentals_df, actual_rebal_date, num_stocks)
        
        if len(selected) == 0:
            print(f"Warning: No stocks selected on {actual_rebal_date}")
            continue
        
        # Allocate equally across selected stocks
        allocation_per_stock = cash / len(selected)
        
        for stock in selected:
            ticker = stock['ticker']
            price = stock['price']
            
            if price > 0:
                shares = allocation_per_stock / price
                holdings[ticker] = shares
                
                transaction_history.append({
                    'date': actual_rebal_date,
                    'action': 'BUY',
                    'ticker': ticker,
                    'shares': shares,
                    'price': price,
                    'value': shares * price
                })
        
        cash = 0.0  # All cash invested
        
        print(f"{strategy_name} - {actual_rebal_date.date()}: Selected {len(selected)} stocks, Total invested: ${total_invested:,.0f}")
    
    # Calculate daily portfolio values
    start_date = pd.Timestamp(rebalance_dates[0])
    
    current_holdings = {}
    current_invested = 0.0
    rebal_idx = 0
    
    for date in all_dates:
        if date < start_date:
            continue
        
        # Check if this is a rebalance date
        if rebal_idx < len(rebalance_dates):
            rebal_date = pd.Timestamp(rebalance_dates[rebal_idx])
            if date >= rebal_date:
                # Get transactions for this date
                date_txns = [t for t in transaction_history 
                            if t['date'] == date or (t['date'] >= rebal_date and t['date'] <= date)]
                
                # Update holdings from transactions
                current_holdings = {}
                for txn in date_txns:
                    if txn['action'] == 'BUY':
                        current_holdings[txn['ticker']] = txn['shares']
                
                current_invested += annual_investment
                rebal_idx += 1
        
        # Calculate portfolio value
        portfolio_value = 0.0
        for ticker, shares in current_holdings.items():
            try:
                price = prices_df.loc[date, ticker]
                if not pd.isna(price):
                    portfolio_value += shares * price
            except:
                pass
        
        portfolio_history.append({
            'date': date,
            'portfolio_value': portfolio_value,
            'total_invested': current_invested,
            'num_holdings': len(current_holdings)
        })
    
    portfolio_df = pd.DataFrame(portfolio_history)
    portfolio_df.set_index('date', inplace=True)
    
    transactions_df = pd.DataFrame(transaction_history)
    
    return portfolio_df, transactions_df

print("Backtest engine defined.")

## Section 4: Run Backtests

In [ ]:
# Run Value-Only Strategy Backtest
print("="*60)
print("Running Value-Only Strategy Backtest")
print("="*60)

value_portfolio, value_transactions = run_backtest(
    strategy_func=select_stocks_value,
    prices_df=all_prices,
    fundamentals_df=fundamentals_df,
    annual_investment=ANNUAL_INVESTMENT,
    num_stocks=NUM_STOCKS,
    strategy_name="Value-Only"
)

print(f"\nValue-Only Final Portfolio Value: ${value_portfolio['portfolio_value'].iloc[-1]:,.2f}")

In [ ]:
# Run Multi-Factor Strategy Backtest
print("="*60)
print("Running Multi-Factor Strategy Backtest")
print("="*60)

multifactor_portfolio, multifactor_transactions = run_backtest(
    strategy_func=select_stocks_multifactor,
    prices_df=all_prices,
    fundamentals_df=fundamentals_df,
    annual_investment=ANNUAL_INVESTMENT,
    num_stocks=NUM_STOCKS,
    strategy_name="Multi-Factor"
)

print(f"\nMulti-Factor Final Portfolio Value: ${multifactor_portfolio['portfolio_value'].iloc[-1]:,.2f}")

In [ ]:
# Calculate SPY Benchmark (DCA $1000/year)
print("="*60)
print("Calculating SPY Benchmark")
print("="*60)

spy_shares = 0.0
spy_invested = 0.0
spy_history = []

for rebal_date_str in REBALANCE_DATES:
    rebal_date = pd.Timestamp(rebal_date_str)
    valid_dates = spy_data.index[spy_data.index >= rebal_date]
    if len(valid_dates) == 0:
        continue
    actual_date = valid_dates[0]
    
    price = spy_data.loc[actual_date]
    new_shares = ANNUAL_INVESTMENT / price
    spy_shares += new_shares
    spy_invested += ANNUAL_INVESTMENT
    print(f"SPY - {actual_date.date()}: Bought {new_shares:.4f} shares @ ${price:.2f}")

# Calculate daily SPY portfolio values
spy_portfolio = pd.DataFrame(index=spy_data.index)
spy_portfolio['portfolio_value'] = 0.0
spy_portfolio['total_invested'] = 0.0

current_shares = 0.0
current_invested = 0.0
rebal_idx = 0

for date in spy_data.index:
    if rebal_idx < len(REBALANCE_DATES):
        rebal_date = pd.Timestamp(REBALANCE_DATES[rebal_idx])
        if date >= rebal_date:
            price = spy_data.loc[date]
            current_shares += ANNUAL_INVESTMENT / price
            current_invested += ANNUAL_INVESTMENT
            rebal_idx += 1
    
    spy_portfolio.loc[date, 'portfolio_value'] = current_shares * spy_data.loc[date]
    spy_portfolio.loc[date, 'total_invested'] = current_invested

# Filter to start date
start_date = pd.Timestamp(REBALANCE_DATES[0])
spy_portfolio = spy_portfolio[spy_portfolio.index >= start_date]

print(f"\nSPY Final Portfolio Value: ${spy_portfolio['portfolio_value'].iloc[-1]:,.2f}")

## Section 5: Performance Metrics & Evaluation

In [ ]:
def calculate_metrics(portfolio_df, name="Strategy"):
    """
    Calculate performance metrics for a portfolio.
    """
    # Total invested
    total_invested = portfolio_df['total_invested'].iloc[-1]
    
    # Final value
    final_value = portfolio_df['portfolio_value'].iloc[-1]
    
    # Total return
    total_return = (final_value - total_invested) / total_invested * 100
    
    # Calculate daily returns
    portfolio_df = portfolio_df.copy()
    portfolio_df['daily_return'] = portfolio_df['portfolio_value'].pct_change()
    
    # CAGR
    years = (portfolio_df.index[-1] - portfolio_df.index[0]).days / 365.25
    if years > 0 and portfolio_df['portfolio_value'].iloc[0] > 0:
        cagr = ((final_value / portfolio_df['portfolio_value'].iloc[0]) ** (1/years) - 1) * 100
    else:
        cagr = 0
    
    # Volatility (annualized)
    daily_vol = portfolio_df['daily_return'].std()
    annual_vol = daily_vol * np.sqrt(252) * 100
    
    # Sharpe Ratio (assuming 4% risk-free rate)
    risk_free_rate = 0.04
    excess_return = (cagr / 100) - risk_free_rate
    sharpe = excess_return / (annual_vol / 100) if annual_vol > 0 else 0
    
    # Max Drawdown
    rolling_max = portfolio_df['portfolio_value'].cummax()
    drawdown = (portfolio_df['portfolio_value'] - rolling_max) / rolling_max
    max_drawdown = drawdown.min() * 100
    
    # Annual returns
    annual_returns = {}
    for year in range(2021, 2026):
        year_start = f"{year}-01-01"
        year_end = f"{year}-12-31"
        year_data = portfolio_df[(portfolio_df.index >= year_start) & (portfolio_df.index <= year_end)]
        if len(year_data) > 1:
            year_return = (year_data['portfolio_value'].iloc[-1] / year_data['portfolio_value'].iloc[0] - 1) * 100
            annual_returns[year] = year_return
    
    return {
        'name': name,
        'total_invested': total_invested,
        'final_value': final_value,
        'total_return': total_return,
        'cagr': cagr,
        'volatility': annual_vol,
        'sharpe_ratio': sharpe,
        'max_drawdown': max_drawdown,
        'annual_returns': annual_returns
    }

# Calculate metrics for all strategies
value_metrics = calculate_metrics(value_portfolio, "Value-Only")
multifactor_metrics = calculate_metrics(multifactor_portfolio, "Multi-Factor")
spy_metrics = calculate_metrics(spy_portfolio, "S&P 500 (SPY)")

print("Metrics calculated.")

In [ ]:
# Display Comparison Table
print("\n" + "="*80)
print("BACKTEST RESULTS COMPARISON")
print("="*80)
print(f"Period: {REBALANCE_DATES[0]} to {value_portfolio.index[-1].date()}")
print(f"Investment: ${ANNUAL_INVESTMENT:,.0f}/year for 5 years = ${ANNUAL_INVESTMENT * 5:,.0f} total")
print("="*80)

comparison_data = {
    'Metric': ['Final Value', 'Total Return', 'CAGR', 'Volatility', 'Sharpe Ratio', 'Max Drawdown'],
    'Value-Only': [
        f"${value_metrics['final_value']:,.2f}",
        f"{value_metrics['total_return']:.1f}%",
        f"{value_metrics['cagr']:.1f}%",
        f"{value_metrics['volatility']:.1f}%",
        f"{value_metrics['sharpe_ratio']:.2f}",
        f"{value_metrics['max_drawdown']:.1f}%"
    ],
    'Multi-Factor': [
        f"${multifactor_metrics['final_value']:,.2f}",
        f"{multifactor_metrics['total_return']:.1f}%",
        f"{multifactor_metrics['cagr']:.1f}%",
        f"{multifactor_metrics['volatility']:.1f}%",
        f"{multifactor_metrics['sharpe_ratio']:.2f}",
        f"{multifactor_metrics['max_drawdown']:.1f}%"
    ],
    'S&P 500 (SPY)': [
        f"${spy_metrics['final_value']:,.2f}",
        f"{spy_metrics['total_return']:.1f}%",
        f"{spy_metrics['cagr']:.1f}%",
        f"{spy_metrics['volatility']:.1f}%",
        f"{spy_metrics['sharpe_ratio']:.2f}",
        f"{spy_metrics['max_drawdown']:.1f}%"
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

In [ ]:
# Portfolio Value Over Time Chart
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Chart 1: Portfolio Value Over Time
ax1 = axes[0, 0]
ax1.plot(value_portfolio.index, value_portfolio['portfolio_value'], label='Value-Only', linewidth=2)
ax1.plot(multifactor_portfolio.index, multifactor_portfolio['portfolio_value'], label='Multi-Factor', linewidth=2)
ax1.plot(spy_portfolio.index, spy_portfolio['portfolio_value'], label='S&P 500 (SPY)', linewidth=2, linestyle='--')
ax1.axhline(y=5000, color='gray', linestyle=':', label='Total Invested ($5,000)')
ax1.set_title('Portfolio Value Over Time', fontsize=12, fontweight='bold')
ax1.set_xlabel('Date')
ax1.set_ylabel('Portfolio Value ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Chart 2: Annual Returns
ax2 = axes[0, 1]
years = list(value_metrics['annual_returns'].keys())
x = np.arange(len(years))
width = 0.25

bars1 = ax2.bar(x - width, [value_metrics['annual_returns'].get(y, 0) for y in years], width, label='Value-Only')
bars2 = ax2.bar(x, [multifactor_metrics['annual_returns'].get(y, 0) for y in years], width, label='Multi-Factor')
bars3 = ax2.bar(x + width, [spy_metrics['annual_returns'].get(y, 0) for y in years], width, label='S&P 500')

ax2.set_title('Annual Returns by Year', fontsize=12, fontweight='bold')
ax2.set_xlabel('Year')
ax2.set_ylabel('Return (%)')
ax2.set_xticks(x)
ax2.set_xticklabels(years)
ax2.legend()
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.grid(True, alpha=0.3, axis='y')

# Chart 3: Drawdown
ax3 = axes[1, 0]
value_dd = (value_portfolio['portfolio_value'] - value_portfolio['portfolio_value'].cummax()) / value_portfolio['portfolio_value'].cummax() * 100
mf_dd = (multifactor_portfolio['portfolio_value'] - multifactor_portfolio['portfolio_value'].cummax()) / multifactor_portfolio['portfolio_value'].cummax() * 100
spy_dd = (spy_portfolio['portfolio_value'] - spy_portfolio['portfolio_value'].cummax()) / spy_portfolio['portfolio_value'].cummax() * 100

ax3.fill_between(value_portfolio.index, value_dd, 0, alpha=0.3, label='Value-Only')
ax3.fill_between(multifactor_portfolio.index, mf_dd, 0, alpha=0.3, label='Multi-Factor')
ax3.plot(spy_portfolio.index, spy_dd, label='S&P 500', linestyle='--', color='green')
ax3.set_title('Drawdown Over Time', fontsize=12, fontweight='bold')
ax3.set_xlabel('Date')
ax3.set_ylabel('Drawdown (%)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Chart 4: Summary Metrics Bar Chart
ax4 = axes[1, 1]
metrics_names = ['CAGR', 'Sharpe Ratio']
value_vals = [value_metrics['cagr'], value_metrics['sharpe_ratio']]
mf_vals = [multifactor_metrics['cagr'], multifactor_metrics['sharpe_ratio']]
spy_vals = [spy_metrics['cagr'], spy_metrics['sharpe_ratio']]

x = np.arange(len(metrics_names))
width = 0.25

ax4.bar(x - width, value_vals, width, label='Value-Only')
ax4.bar(x, mf_vals, width, label='Multi-Factor')
ax4.bar(x + width, spy_vals, width, label='S&P 500')

ax4.set_title('Key Performance Metrics', fontsize=12, fontweight='bold')
ax4.set_xticks(x)
ax4.set_xticklabels(metrics_names)
ax4.legend()
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('backtest-charts.png', dpi=150, bbox_inches='tight')
plt.show()

print("Charts saved to backtest-charts.png")

## Section 6: Investment Recommendations

In [ ]:
# Determine winner and generate recommendations
strategies = [
    ('Value-Only', value_metrics),
    ('Multi-Factor', multifactor_metrics),
    ('S&P 500 (SPY)', spy_metrics)
]

# Sort by CAGR
by_cagr = sorted(strategies, key=lambda x: x[1]['cagr'], reverse=True)
# Sort by Sharpe
by_sharpe = sorted(strategies, key=lambda x: x[1]['sharpe_ratio'], reverse=True)
# Sort by max drawdown (less negative is better)
by_drawdown = sorted(strategies, key=lambda x: x[1]['max_drawdown'], reverse=True)

winner_cagr = by_cagr[0][0]
winner_sharpe = by_sharpe[0][0]
winner_drawdown = by_drawdown[0][0]

print("\n" + "="*80)
print("INVESTMENT RECOMMENDATIONS BASED ON BACKTEST")
print("="*80)

print("\n### Strategy Selection")
print(f"- Best CAGR: {winner_cagr} ({by_cagr[0][1]['cagr']:.1f}%)")
print(f"- Best Risk-Adjusted (Sharpe): {winner_sharpe} ({by_sharpe[0][1]['sharpe_ratio']:.2f})")
print(f"- Lowest Drawdown: {winner_drawdown} ({by_drawdown[0][1]['max_drawdown']:.1f}%)")

# Compare active strategies to benchmark
value_vs_spy = value_metrics['cagr'] - spy_metrics['cagr']
mf_vs_spy = multifactor_metrics['cagr'] - spy_metrics['cagr']

print("\n### Alpha vs Benchmark")
print(f"- Value-Only vs S&P 500: {value_vs_spy:+.1f}% CAGR difference")
print(f"- Multi-Factor vs S&P 500: {mf_vs_spy:+.1f}% CAGR difference")

print("\n### Implementation Guidelines")
if value_metrics['sharpe_ratio'] > multifactor_metrics['sharpe_ratio']:
    print("1. Value-Only strategy shows better risk-adjusted returns")
    print("   - Simpler to implement (fewer factors to track)")
    print("   - Consider this as primary approach")
else:
    print("1. Multi-Factor strategy shows better risk-adjusted returns")
    print("   - More complex but captures multiple alpha sources")
    print("   - Consider this as primary approach")

print("2. Rebalance annually in early January")
print("3. Maintain diversification with 25 stock positions")
print(f"4. Expected realistic returns: {min(value_metrics['cagr'], multifactor_metrics['cagr']):.0f}-{max(value_metrics['cagr'], multifactor_metrics['cagr']):.0f}% CAGR")

print("\n### Risk Considerations")
print("- Past performance does not guarantee future results")
print("- Survivorship bias may inflate historical returns (we only tested stocks that exist today)")
print("- Consider tax implications of annual rebalancing")
print("- Transaction costs not included in this backtest")
print(f"- Maximum drawdown of {min(value_metrics['max_drawdown'], multifactor_metrics['max_drawdown']):.1f}% requires psychological preparation")

## Section 7: Export Results to Excel

In [ ]:
# Export detailed results to Excel
with pd.ExcelWriter('backtest-results.xlsx', engine='openpyxl') as writer:
    # Summary sheet
    summary_data = {
        'Metric': ['Total Invested', 'Final Value', 'Total Return (%)', 'CAGR (%)', 
                   'Volatility (%)', 'Sharpe Ratio', 'Max Drawdown (%)'],
        'Value-Only': [value_metrics['total_invested'], value_metrics['final_value'],
                       value_metrics['total_return'], value_metrics['cagr'],
                       value_metrics['volatility'], value_metrics['sharpe_ratio'],
                       value_metrics['max_drawdown']],
        'Multi-Factor': [multifactor_metrics['total_invested'], multifactor_metrics['final_value'],
                         multifactor_metrics['total_return'], multifactor_metrics['cagr'],
                         multifactor_metrics['volatility'], multifactor_metrics['sharpe_ratio'],
                         multifactor_metrics['max_drawdown']],
        'S&P 500 (SPY)': [spy_metrics['total_invested'], spy_metrics['final_value'],
                          spy_metrics['total_return'], spy_metrics['cagr'],
                          spy_metrics['volatility'], spy_metrics['sharpe_ratio'],
                          spy_metrics['max_drawdown']]
    }
    pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)
    
    # Annual returns
    annual_data = {
        'Year': list(value_metrics['annual_returns'].keys()),
        'Value-Only (%)': list(value_metrics['annual_returns'].values()),
        'Multi-Factor (%)': [multifactor_metrics['annual_returns'].get(y, 0) for y in value_metrics['annual_returns'].keys()],
        'S&P 500 (%)': [spy_metrics['annual_returns'].get(y, 0) for y in value_metrics['annual_returns'].keys()]
    }
    pd.DataFrame(annual_data).to_excel(writer, sheet_name='Annual Returns', index=False)
    
    # Daily portfolio values
    combined_portfolio = pd.DataFrame({
        'Value-Only': value_portfolio['portfolio_value'],
        'Multi-Factor': multifactor_portfolio['portfolio_value'],
        'S&P 500': spy_portfolio['portfolio_value']
    })
    combined_portfolio.to_excel(writer, sheet_name='Daily Values')
    
    # Transactions
    value_transactions.to_excel(writer, sheet_name='Value-Only Transactions', index=False)
    multifactor_transactions.to_excel(writer, sheet_name='Multi-Factor Transactions', index=False)

print("Results exported to backtest-results.xlsx")

In [ ]:
print("\n" + "="*80)
print("BACKTEST COMPLETE")
print("="*80)
print(f"\nOutputs generated:")
print(f"  - backtest-results.xlsx (detailed results)")
print(f"  - backtest-charts.png (visualization)")
print(f"\nKey Findings:")
print(f"  - Value-Only final value: ${value_metrics['final_value']:,.2f} ({value_metrics['total_return']:.1f}% return)")
print(f"  - Multi-Factor final value: ${multifactor_metrics['final_value']:,.2f} ({multifactor_metrics['total_return']:.1f}% return)")
print(f"  - S&P 500 final value: ${spy_metrics['final_value']:,.2f} ({spy_metrics['total_return']:.1f}% return)")